In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/machinelearningproject/modified_dataset.csv
/kaggle/input/image1/medicine2 (1).jpg
/kaggle/input/dataset1/a-z.csv


In [6]:
import pandas as pd
#Load the dataset
dataset = pd.read_csv('/kaggle/input/dataset1/a-z.csv')

# List of columns to drop
columns_to_drop = ['Is_discontinued', 'manufacturer_name', 'type','pack_size_label']

# Droping the specified columns from the DataFrame
dataset.drop(columns=columns_to_drop, inplace=True)

# To display the modified DataFrame
dataset.head()

,id,name,price(₹),short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,Pheniramine (25mg),NaN


In [7]:
import pandas as pd
df = dataset
df

,id,name,price(₹),short_composition1,short_composition2
0,1,Augmentin 625 Duo Tablet,223.42,Amoxycillin (500mg),Clavulanic Acid (125mg)
1,2,Azithral 500 Tablet,132.36,Azithromycin (500mg),NaN
2,3,Ascoril LS Syrup,118.00,Ambroxol (30mg/5ml),Levosalbutamol (1mg/5ml)
3,4,Allegra 120mg Tablet,218.81,Fexofenadine (120mg),NaN
4,5,Avil 25 Tablet,10.96,Pheniramine (25mg),NaN
...,...,...,...,...,...
253968,253969,Ziyapod 100mg Oral Suspension,62.30,Cefpodoxime Proxetil (100mg),NaN
253969,253970,Zemhart 30mg Tablet,54.00,Diltiazem (30mg),NaN
253970,253971,Zivex 25mg Tablet,57.00,Hydroxyzine (25mg),NaN
253971,253972,ZI Fast 500mg Injection,152.00,Azithromycin (500mg),NaN


In [4]:
df1=df['name']
df1

0              Augmentin 625 Duo Tablet
1                   Azithral 500 Tablet
2                      Ascoril LS Syrup
3                  Allegra 120mg Tablet
4                        Avil 25 Tablet
                      ...              
253968    Ziyapod 100mg Oral Suspension
253969              Zemhart 30mg Tablet
253970                Zivex 25mg Tablet
253971          ZI Fast 500mg Injection
253972        Zyvocol 1% Dusting Powder
Name: name, Length: 253973, dtype: object

In [5]:
x = df.drop("id", axis=1)

In [91]:
price = x.drop(['name', 'short_composition1', 'short_composition2'], axis='columns')
#price

In [92]:
text = x.drop("price(₹)", axis=1)

In [93]:
df['combined_text'] = text[['name', 'short_composition1', 'short_composition2']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
#df['combined_text'] 

In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(words):
    words = words.lower() 
    words = words.translate(str.maketrans('', '', string.punctuation)) 
    tokens = word_tokenize(words) 
    stop_words = set(stopwords.words('english')) 
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    processed_text = ' '.join(tokens) 
    return processed_text

df['combined_text'] = df['combined_text'].apply(preprocess_text)


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
scale = MaxAbsScaler()
matrix_scaled = scale.fit_transform(matrix)
#print(matrix_scaled)

In [ ]:
#converting the scaled values to pandas dataframe
import pandas as pd
from scipy.sparse import csr_matrix

dense = pd.DataFrame.sparse.from_spmatrix(matrix_scaled )
#dense


In [ ]:
# getting the top 5000 features to reduce the complexity
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
selector.fit(dense)
top_5000_indices = selector.variances_.argsort()[-5000:][::-1]
reduced_matrix = dense.iloc[:, top_5000_indices]

In [ ]:
#reduced_matrix

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

num_cols = reduced_matrix.shape[1]
random_names = [f"col_{i}" for i in range(num_cols)]
reduced_matrix.columns = random_names
#print(reduced_matrix)

In [ ]:
merged = pd.concat([reduced_matrix,price],axis='columns')
#merged

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=400, random_state=42)
cluster_labels = kmeans.fit_predict(reduced_matrix)
merged['cluster'] = cluster_labels
#print("Cluster Centroids:")
#print(kmeans.cluster_centers_)

In [ ]:
#cluster_labels

In [ ]:
import pandas as pd

# Assuming df and cluster_labels are defined elsewhere
cluster_data1 = pd.DataFrame({'Medicine_name': df['name'].str.lower(), 'Cluster': cluster_labels})
#print(cluster_data1)


In [ ]:
import cv2
import numpy as np
import pytesseract

# Path to the handwritten prescription image (replace 'handwritten_prescription.jpg' with your image file path)
image_path = '//kaggle/input/image1/medicine2 (1).jpg'

# Read the handwritten prescription image
image = cv2.imread(image_path)

# Check if the image is successfully loaded
if image is None:
    print("Error: Unable to read the image file.")
else:
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to remove noise and smooth the image
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

    # Apply adaptive thresholding to binarize the image
    _, thresh_image = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    # Invert the binary image
    inverted_image = cv2.bitwise_not(thresh_image)

    # Perform OCR on the inverted binary image
    extracted_text = pytesseract.image_to_string(inverted_image, config='--psm 6')

    # Print the extracted text
    #print("Extracted Text:")
    #print(extracted_text)

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
cluster_data = pd.read_csv('/kaggle/input/machinelearningproject/modified_dataset.csv')

# Print unique values in the 'name' column
print("Unique values in the 'name' column:")
print(cluster_data['name'].unique())
# Split the extracted text into words and convert them to lowercase
extracted_words = extracted_text.lower().split()

# Print the extracted words
#print("Extracted words:")
#print(extracted_words)


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
cluster_data = pd.read_csv('/kaggle/input/machinelearningproject/modified_dataset.csv')

# Convert the 'name' column to lowercase and split it into words
split_lower_names = cluster_data['name'].str.lower().str.split()

#print(split_lower_names)


In [ ]:
# Flatten the list of lists into a single list
flat_split_lower_names = [word for sublist in split_lower_names for word in sublist]

# Convert the flattened list to a set
split_lower_names_set = set(flat_split_lower_names)

# Convert the extracted words list to a set
extracted_words_set = set(extracted_words)

# Find the intersection of the two sets
common_words = split_lower_names_set.intersection(extracted_words_set)

# Print the result
if common_words:
    # print("Common words found:", common_words)
    pass
else:
    # print("No common words found.")
    pass


In [ ]:
# Column name for the output
column_name = "Names with common words as the starting word:"

# Initialize a list to store names with common words as the starting word
names_with_common_words_starting_word = []

# Iterate through each sublist in split_lower_names
for sublist in split_lower_names:
    # Check if the first word in the sublist is present in common_words and its length is greater than 1
    if sublist[0] in common_words and len(sublist[0]) > 2:
        # If a common word is found as the starting word and it's not a single letter, add the sublist to names_with_common_words_starting_word
        names_with_common_words_starting_word.append(' '.join(sublist))

# Check if any names contain common words as the starting word
if names_with_common_words_starting_word:
    #print(column_name)
     for name in names_with_common_words_starting_word:
        #print(name)
        pass
else:
    #print("No common words found as the starting word in any names.")
    pass
    


In [ ]:
import pandas as pd

def suggest_alternative_medicines(prescribed_medicine, cluster_data):
    prescribed_cluster = cluster_data1.loc[cluster_data1['Medicine_name'] == prescribed_medicine, 'Cluster'].iloc[0]
    alternatives = cluster_data1.loc[cluster_data1['Cluster'] == prescribed_cluster, 'Medicine_name'].tolist()
    alternatives.remove(prescribed_medicine)  
    return alternatives[:10]



# Example usage
if names_with_common_words_starting_word:
    prescribed_medicine = names_with_common_words_starting_word[0]
    alternatives = suggest_alternative_medicines(prescribed_medicine, cluster_data1)
    print("Alternative medicines for", prescribed_medicine, ":", alternatives)
else:
    print("No common words found as the starting word in any names.")


In [ ]:
import webbrowser

# Define the links
links = ["https://www.netmeds.com/", "https://www.1mg.com/", "https://www.apollopharmacy.in/"]

# Print and open the links
for link in links:
    print(f"Click to open: {link}")
    webbrowser.open(link)